In [1]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
# os.chdir(os.path.join(base_dir, 'serverproject'))
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss
import time

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject


In [2]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData, FastRecapData

from core import services
from core import utils
from core import controller
from dataclasses import asdict


In [3]:
# video_id="phITCK1LnDs"
video_id="n-40IAey5eo"
cost=0

In [4]:
# Get and process transcript
t_transcript=time.time()
# times.append({"transcript":t_transcript})
await services.video_and_transcript.download_youtube_transcript(video_id)
raw_transcript=await services.video_and_transcript.read_and_process_youtube_transcript(video_id)
transcript, linked_transcript=services.fast.transcript_processing.process_yt_transcript(raw_transcript, video_id)
if (len(utils.api_requests.enc.encode(transcript)))>(120*1000):
    raise Exception("Transcript too long")
print("TRANSCRIPT TIME: ", time.time()-t_transcript)

download thread closed
d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi\serverproject
TRANSCRIPT TIME:  5.760482311248779


In [ ]:
plot_object=await controller.FastPlotController.generation_step(transcript, linked_transcript, video_id)

In [5]:
# import deepcopy
from copy import deepcopy
plot_object_copy=deepcopy(plot_object)

NameError: name 'plot_object' is not defined

In [ ]:
topics_str, major_topics, temp_cost=await services.fast.data_gen.create_topics_split(transcript, utils.ModelNameEnum.llama_3_2_90B_vision_preview)

In [ ]:
plot_object=await controller.FastPlotController.generation_step(transcript, linked_transcript, video_id)

In [ ]:
fast_recap_filter=await sync_to_async(FastRecapData.objects.filter)(video_id=video_id)
fast_recap_data=await sync_to_async(fast_recap_filter.first)()
if fast_recap_data==None:
    fast_recap_data=FastRecapData(video_id=video_id)
    
fast_recap_data.plot_object=asdict(plot_object)
await sync_to_async(fast_recap_data.save)()


# PART BY PART

In [6]:
times=[]

In [7]:
topics_str, major_topics, temp_cost=await services.fast.data_gen.create_topics_split(transcript, utils.ModelNameEnum.llama_3_2_90B_vision_preview)

SUMMARIZED:  3.6633734703063965
Topics Done:  2.6140997409820557


In [8]:
model_select=utils.ModelNameEnum.llama_3_70b_8192


In [11]:
text_chunk_batches[0]

["oh you know what else Chris yes um I Iprobably would never do this on apodcast but I made a couple notes ofthings I'm going to say because yeahwell because you know you're probablyone of the most people that you kind ofremind me of you know some of my likeclosest friends in home word it's alwayslike trying to get to the bottom ofsomething you know what I mean so I feltlike I have a good couple good theoriesthat oh I [\\h__\\h] love this you cameprepared yeah also well we've talkedabout this already but I said your stuffthat you post a lot of times feels verymuch directed for me like I'm the exactlike you know I'm the one targetaudience being seen what was the whatwas one of the ones that that's happenedwith probably the most ever is cuz theidea that you know hard work you can'talways just like hard work your way outof a problem and I think you know youbecomeso accustomed to this is how I you knowExcel over people and then you're likewell everyone's talented at a certainlevel and then

In [9]:
# Annotate Text batches
text_chunks_no_overlap = await services.stream_plot.data_gen.create_text_chunks(transcript, 0)
text_chunk_batches = await services.stream_plot.data_gen.generate_text_chunk_batches(text_chunks_no_overlap)
responses, annotated_results, temp_cost = await services.fast.data_gen.annotate_all_batches(text_chunk_batches, topics_str, model_select)
cost+=temp_cost
t_annotations=time.time()
times.append({"annotations":t_annotations})
print("Annotated Text Batches")


Cost:  0.0032536600000000002
Cost:  0.00324619
Cost:  0.0032880999999999995
Cost:  0.00332877
Cost:  0.00327556
Cost:  0.00328967
Cost:  0.00332724
Cost:  0.0032581399999999996
Cost:  0.00296742
Results:  18
Results:  18
Results:  18
Results:  18
Results:  18
Results:  18
Results:  18
Results:  18
Results:  17
Annotated Text Batches


In [15]:
text_chunk_batches

[["oh you know what else Chris yes um I Iprobably would never do this on apodcast but I made a couple notes ofthings I'm going to say because yeahwell because you know you're probablyone of the most people that you kind ofremind me of you know some of my likeclosest friends in home word it's alwayslike trying to get to the bottom ofsomething you know what I mean so I feltlike I have a good couple good theoriesthat oh I [\\h__\\h] love this you cameprepared yeah also well we've talkedabout this already but I said your stuffthat you post a lot of times feels verymuch directed for me like I'm the exactlike you know I'm the one targetaudience being seen what was the whatwas one of the ones that that's happenedwith probably the most ever is cuz theidea that you know hard work you can'talways just like hard work your way outof a problem and I think you know youbecomeso accustomed to this is how I you knowExcel over people and then you're likewell everyone's talented at a certainlevel and the

In [20]:
# single annotation
resp, resp_cost=await services.fast.data_gen.annotate_batch(text_chunk_batches[0], topics_str, model_name=utils.ModelNameEnum.llama_3_2_11b_vision_preview)

Cost:  0.004916360000000001


In [21]:
len(resp)

4828

In [ ]:
# Create Plot
annotated_segments, category_locations = await services.stream_plot.data_processing.create_segments(linked_transcript, annotated_results, major_topics, transcript)
plot_segments=await services.stream_plot.data_processing.annotated_to_plot_segments(annotated_segments)
plot_object=await services.stream_plot.data_processing.create_plot_object(plot_segments, category_locations, video_id)
t_plot_object=time.time()
times.append({"plot_object":t_plot_object})
print("Created Plot Object")

# TESTING

In [ ]:
system_prompt="Summarize everything in 2 sentences"
# user_prompt=transcript[:80000]
user_prompt=transcript[:80000]

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]
resp, co=await utils.async_response_handler(prompt, utils.ModelNameEnum.llama_3_1_70B_versatile)

In [ ]:
print(resp)

In [ ]:
system_prompt="Summarize everything in 2 sentences"
user_prompt=transcript[:80000]
# user_prompt=transcript[:8000]

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]
resp, co=await utils.async_response_handler(prompt, utils.ModelNameEnum.gpt_4o_mini)

In [ ]:
system_prompt="Summarize everything in 2 sentences"
user_prompt=transcript[:50000]
# user_prompt=transcript[:8000]

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]
resp, co=await utils.api_requests.streamed_response_handler(prompt, utils.ModelNameEnum.llama_3_1_8B_instant)

In [ ]:
system_prompt="Summarize everything in 2 sentences"
user_prompt=transcript[:80000]
# user_prompt=transcript[:8000]

prompt=[{"role":"system", "content":system_prompt}, {"role":"user", "content":user_prompt}]
resp, co=await utils.api_requests.streamed_response_handler(prompt, utils.ModelNameEnum.llama_3_2_11b_vision_preview)